<a href="https://colab.research.google.com/github/Nikil263/Movie-Review-Tweets-Dataset-for-Spoiler-Detection/blob/main/Baseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline 2

In [1]:
import pandas as pd

In [2]:
df3 = pd.read_csv('/content/labeldataset mod-2.csv')
labeled_tweets = df3.iloc[:500,:]
val = df3.iloc[800:1001,:]
train1 = df3.iloc[1001:1501,:]
val1 = df3.iloc[1801:2001,:]
train2 = df3.iloc[2001:2501,:]
val2 = df3.iloc[2801:3001,:]
train3 = df3.iloc[3001:3501,:]
val3 = df3.iloc[3801:4001,:]
unlabeled_tweets = df3.iloc[500:800,:]
unlabeled_tweets1 = df3.iloc[1501:1801,:]
unlabeled_tweets2 = df3.iloc[2501:2801,:]
unlabeled_tweets3 = df3.iloc[3501:3801,:]
unlabeled_tweets = unlabeled_tweets.append(unlabeled_tweets1)
unlabeled_tweets = unlabeled_tweets.append(unlabeled_tweets2)
unlabeled_tweets = unlabeled_tweets.append(unlabeled_tweets3)

labeled_tweets = labeled_tweets.append(train1)
labeled_tweets = labeled_tweets.append(train2)
labeled_tweets = labeled_tweets.append(train3)
val = val.append(val1)
val = val.append(val2)
val = val.append(val3)

<ipython-input-2-cd87a4432836>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unlabeled_tweets = unlabeled_tweets.append(unlabeled_tweets1)
<ipython-input-2-cd87a4432836>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unlabeled_tweets = unlabeled_tweets.append(unlabeled_tweets2)
<ipython-input-2-cd87a4432836>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unlabeled_tweets = unlabeled_tweets.append(unlabeled_tweets3)
<ipython-input-2-cd87a4432836>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labeled_tweets = labeled_tweets.append(train1)
<ipython-input-2-cd87a4432836>:19: FutureWarning: The frame.append method is deprecated

In [3]:
del unlabeled_tweets['LABEL']
unlabeled_tweets['LABEL']=''
unlabeled_tweets



,TWEET,LABEL
500,Associate administrator… The top dog… William ...,
501,Her Former boss… Rachel…https://t.co/WaWMGkVweO,
502,@koyushii i want to know more about the space ...,
503,nGen integrated cargo management system replac...,
504,Divergent economic engagement with Ethiopia by...,
...,...,...
3796,@RagnerLoL Well done guyz !,
3797,@KatanaHugo Great photo!🤩🤩\nGood evening Hugo....,
3798,I’m not even packing my things,
3799,Fix location option issue from [ grafana/world...,


In [4]:
pip install semisupervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for semisupervised: filename=semisupervised-0.0.28-py3-none-any.whl size=21483 sha256=2125bab54e9996dbc3203a17483789ca659ccdbe38fa72f41c051e32ab6261f5
  Stored in directory: /root/.cache/pip/wheels/a2/8e/5f/436fcb0f5f31e029f5d3c6310282e2d324796acdcb99b8c658
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=060afbdf20596af012b3467bdb7394bf07366cace543752518d9e5dbb1b92899
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built semisupervised sklearn


In [5]:
pip show semisupervised


Name: semisupervised
Version: 0.0.28
Summary: Python implementation of semi-supervised learning algorithm
Home-page: https://github.com/rosefun/semisupervised
Author: Rosefun
Author-email: rosefun@foxmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.9/dist-packages
Requires: numpy, pandas, sklearn
Required-by: 


In [6]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=f258615ee438b64d8d28c4a2c979c6b46450aa416eeaeb15a8b7756e76968a40
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [13]:
import re
import string
import nltk
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import LinearSVC

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
# Assuming you have a labeled dataset of tweets in a variable called 'labeled_tweets'
# and an unlabeled dataset of tweets in a variable called 'unlabeled_tweets'

# Step 1: Define a function to preprocess the tweet text data
def preprocess_tweet_text(tweet):
    
    tweet = emoji.replace_emoji(tweet,'')
    # Remove URLs
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Remove punctuations
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    
    # Tokenize the tweet text
    tokens = word_tokenize(tweet)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the tokens back into a string
    processed_tweet = " ".join(tokens)
    
    return processed_tweet

# Step 2: Apply the preprocess_tweet_text function to the tweet text columns of both labeled and unlabeled datasets
labeled_tweets['TWEET'] = labeled_tweets['TWEET'].apply(preprocess_tweet_text)
unlabeled_tweets['TWEET'] = unlabeled_tweets['TWEET'].apply(preprocess_tweet_text)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object to convert tweet texts into feature vectors
vectorizer = CountVectorizer()

# Convert the labeled and unlabeled tweet texts into feature vectors
labeled_features = vectorizer.fit_transform(labeled_tweets['TWEET'])
unlabeled_features = vectorizer.transform(unlabeled_tweets['TWEET'])

# Initialize the classifier with the labeled data
clf = SelfTrainingClassifier(LinearSVC(), threshold=0.8)
clf.fit(labeled_features, labeled_tweets['LABEL'])



# Step 4: Loop through a fixed number of iterations or until all tweets are labeled
num_iterations = 0
while num_iterations < 800 and len(unlabeled_tweets) > 0:
    # Step 4a: Predict labels for the unlabeled tweets using the current classifier
    predicted_labels = clf.predict(unlabeled_features)
    
    # Step 4b: Identify tweets with high confidence (probability > threshold)
    high_confidence_mask = predicted_labels != -1
    
    # Step 4c: Add high-confidence tweets to the labeled dataset and remove them from the unlabeled dataset
    labeled_tweets = labeled_tweets.append(unlabeled_tweets[high_confidence_mask])
    labeled_tweets.reset_index(inplace=True, drop=True)
    unlabeled_tweets = unlabeled_tweets[~high_confidence_mask]
    unlabeled_tweets.reset_index(inplace=True, drop=True)
    
    # Step 4d: Retrain the classifier with the updated labeled dataset
    clf.fit(labeled_features, labeled_tweets['LABEL'])
    
    # Step 5: Increment the iteration counter
    num_iterations += 1

    

/usr/local/lib/python3.9/dist-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


In [15]:
# Convert the test set tweet texts into feature vectors using the same vectorizer object
test_features = vectorizer.transform(val['TWEET'])

# Use the trained classifier to predict labels for the test set
val['prediction'] = clf.predict(test_features)


In [16]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the predicted labels
accuracy = accuracy_score(val['LABEL'], val['prediction'])

print(f"Accuracy: {accuracy}")


Accuracy: 0.775
